In [1]:
%matplotlib inline
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
from pylab import *
import matplotlib 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import MultipleLocator
from sklearn import linear_model

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.kernel_ridge import KernelRidge
from sklearn.naive_bayes import GaussianNB
from collections import OrderedDict
from sklearn import mixture
from sklearn import cluster, preprocessing, mixture 
#　データセット読み込み
df = pd.read_csv('Perovskite_Water_Splitting_Dup.csv')
df = df.rename(columns={'B1_mole': 'B1 mole'})
df_per = pd.read_csv('periodictable.csv')

In [21]:
df.iloc[:,1:8] = df.iloc[:,1:8].fillna(value = 'non')#.drop_duplicates()
df["Label"] = df.iloc[:,1:8].sum(axis = 1)

df["Label"] = [a.replace("non","") for a in df["Label"]]

df

,Ref,A,A1,A2,B,B1,B2,X,X1,A mole,A1 mole,A2 mole,B mole,B1 mole,B2 mole,X mole,X1 mole,Prep Meth,CalcT,Calc,Prom Meth,Promoter,Prom,Crystal,BandGap,Surface Area,CatW,Alcohol%,Sac. Agent1,Sac. Agent2,Light,Power,Wave,ProdRate,Label
0,[S1],Sr,non,non,Ti,non,non,O,NaN,0.478,NaN,NaN,0.261,NaN,NaN,0.262,NaN,flux,1373.0,10.0,impreg,Rh,0.1,cubic,3.20,0.9,1.0,0.0,NaN,NaN,Xe,300.0,300.0,540.00,SrTiO
1,[S1],Sr,non,non,Ti,non,non,O,NaN,0.478,NaN,NaN,0.261,NaN,NaN,0.262,NaN,flux,1273.0,10.0,impreg,Rh,0.1,cubic,3.20,1.3,1.0,0.0,NaN,NaN,Xe,300.0,300.0,540.00,SrTiO
2,[S1],Sr,non,non,Ti,non,non,O,NaN,0.478,NaN,NaN,0.261,NaN,NaN,0.262,NaN,flux,1173.0,10.0,impreg,Rh,0.1,cubic,3.20,1.9,1.0,0.0,NaN,NaN,Xe,300.0,300.0,32.00,SrTiO
3,[S1],Sr,non,non,Ti,Al,non,O,NaN,0.477,NaN,NaN,0.261,0.0001,NaN,0.262,NaN,flux,1273.0,10.0,impreg,Rh,0.1,cubic,3.20,0.9,1.0,0.0,NaN,NaN,Xe,300.0,300.0,40.00,SrTiAlO
4,[S1],Sr,non,non,Ti,Al,non,O,NaN,0.477,NaN,NaN,0.260,0.0010,NaN,0.261,NaN,flux,1273.0,10.0,impreg,Rh,0.1,cubic,3.20,2.7,1.0,0.0,NaN,NaN,Xe,300.0,300.0,380.00,SrTiAlO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,[S151],La,non,non,Co,non,non,O,NaN,0.565,NaN,NaN,0.240,NaN,NaN,0.195,NaN,SSR,1073.0,2.0,NaN,NaN,NaN,rhombohedral,1.42,NaN,1.0,0.0,CH2O,NaN,Xe,500.0,NaN,14.37,LaCoO
536,[S151],La,non,non,Co,non,non,O,NaN,0.565,NaN,NaN,0.240,NaN,NaN,0.195,NaN,SSR,773.0,2.0,NaN,Au,0.1,rhombohedral,1.42,NaN,1.0,0.0,CH2O,NaN,Xe,500.0,NaN,22.75,LaCoO
537,[S151],La,non,non,Co,non,non,O,NaN,0.565,NaN,NaN,0.240,NaN,NaN,0.195,NaN,SSR,773.0,2.0,NaN,Au,0.5,rhombohedral,1.42,NaN,1.0,0.0,CH2O,NaN,Xe,500.0,NaN,33.53,LaCoO
538,[S151],La,non,non,Co,non,non,O,NaN,0.565,NaN,NaN,0.240,NaN,NaN,0.195,NaN,SG,773.0,2.0,NaN,Au,1.0,rhombohedral,1.42,NaN,1.0,0.0,CH2O,NaN,Xe,500.0,NaN,28.74,LaCoO


In [20]:
df2

,A,A1,A2,B,B1,B2,X,Label
0,Sr,non,non,Ti,non,non,O,SrTiO
1,Sr,non,non,Ti,non,non,O,SrTiO
2,Sr,non,non,Ti,non,non,O,SrTiO
3,Sr,non,non,Ti,Al,non,O,SrTiAlO
4,Sr,non,non,Ti,Al,non,O,SrTiAlO
...,...,...,...,...,...,...,...,...
535,La,non,non,Co,non,non,O,LaCoO
536,La,non,non,Co,non,non,O,LaCoO
537,La,non,non,Co,non,non,O,LaCoO
538,La,non,non,Co,non,non,O,LaCoO
